### Imports

In [1]:
import os
import h5py as h5
import json

import numpy as np  # arrays
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# local imports (found in this repo)
from utils import *
from animator import SacSacAnimator

### Activate interactive plotting
By default, inline plots are static. Here we specify one of two options (comment out the undesired command) that will open plots with GUI controls for us.
- **qt ->** figures opened in windows outside the notebook
- **notebook ->** figures within notebook underneath generating cell.

In [2]:
# %matplotlib qt 
%matplotlib notebook

### Load existing data, or run NEURON model to generate new data
Parameters can be provided to the `SacPair` class in the form of a dict which will be used to overwrite the default properties of the `Sac` objects generated (see `sac_pair.py`).

In [3]:
base_path = "/mnt/Data/NEURONoutput/sac_sac/"
data_path = base_path + "base_run/"
save_name = "test"
load_name = "base"
# load_name = None

if load_name is not None:
    with h5.File(os.path.join(data_path, load_name + ".h5"), "r") as f:
        data = unpack_hdf(f)
else:
    import sac_pair
    sac_params = {
#         "bp_props": {
#             "prox": {"tau1": 10, "tau2": 200, "rev": 0, "weight": .000275, "delay": 0},
#             "dist": { "tau1": .1, "tau2": 20, "rev": 0, "weight": .000495, "delay": 0}
#         },
        "bp_props": {
            "prox": {"tau1": 10, "tau2": 200, "rev": 0, "weight": .0003, "delay": 0},
            "dist": { "tau1": .1, "tau2": 25, "rev": 0, "weight": .0003, "delay": 0}
        },
    }
    model = sac_pair.SacPair(sac_params=sac_params)
    runner = sac_pair.Runner(model, data_path=data_path)
    data = runner.velocity_mechanism_run(
        velocities=[.1, .25, .5, .75, 1, 1.25, 1.5, 1.75, 2, 3, 4],
        uniform_props={"tau1": 1, "tau2": 25},
        n_trials=5
    )
    pack_hdf(data_path + save_name, data)

model_params = json.loads(data["control"]["model_params"])
exp_params = json.loads(data["control"]["exp_params"])
exps = {k: v["data"] for k, v in data.items()}

### Common axes, labels, and parameters generated or pulled from the control experiment data which apply to all conditions

In [4]:
rec_xaxis = np.arange(0, exp_params["tstop"] + exp_params["dt"], exp_params["dt"])
velocities = exp_params["velocities"]

### Experiment dict contents / shapes

In [5]:
print("Electrode recording dict keys:", [k for k in exps["control"]["soma"]["a"].keys()])
print("Synapse recording dict keys:", [k for k in exps["control"]["gaba"]["a"].keys()])
print("Recording shape:", exps["control"]["soma"]["a"]["v"].shape)

Electrode recording dict keys: ['ica', 'v']
Synapse recording dict keys: ['g', 'i']
Recording shape: (5, 9, 4001)


### Model Schematic and Animation
Colours are normalized to the maximum values of their corresponding measures across all velocities and both SACs in a particular condition. Thus, for the matching visible components (e.g. 2nd dist BPCs or terminals), the colour scaling is shared.

- **soma:** membrane voltage at centre
- **terminal:** membrane voltage at tip (GABA pre-synapse)
- **bipolars:** conductance
- **GABA:** conductance

In [6]:
sac_anim = SacSacAnimator(exps, exp_params, model_params, reveal_time=True)
sac_anim_fig, sac_anim_ax = sac_anim.build_animation_fig(figsize=(8, 12))

<IPython.core.display.Javascript object>

AttributeError: 'AxesSubplot' object has no attribute 'set_xlabels'

### PN DSi measurements
Here the bar is simply moving in the same direction, over SAC A then SAC B, but we can use the signals in the opposing SACs to calculate a preferred minus null direction selective index.

- **peak:** Using the deflection from the minimum voltage
- **thresh_area:** Area under the voltage above a specified threshold
- **iCa**: Area of calcium current (polarity flipped to positive)
- **gaba:** Area of GABA conductance *applied* the *opposing* SAC  

In [ ]:
thresh = -55
dsis = {
    cond: {
        "peak": pn_dsi(
            peak_vm_deflection(exp["term"]["a"]["v"]),
            peak_vm_deflection(exp["term"]["b"]["v"])
        ),
        "thresh_area": pn_dsi(
            thresholded_area(exp["term"]["a"]["v"], thresh),
            thresholded_area(exp["term"]["b"]["v"], thresh)
        ),
        "iCa": pn_dsi(
            np.sum(exp["term"]["a"]["ica"] * -1, axis=-1),
            np.sum(exp["term"]["b"]["ica"] * -1, axis=-1)
        ),
        "gaba": pn_dsi(
            np.sum(exp["gaba"]["b"]["g"], axis=-1), 
            np.sum(exp["gaba"]["a"]["g"], axis=-1)
        )
    }
    for cond, exp in exps.items()
}

### Velocity Tuning of PN DSi
The above measurements of PN DSi for each of the experimental conditions.
- **control:** prox-distal bipolar time constants and reciprocal GABA are intact
- **no_gaba:** bipolar mechanism remains, while GABA synapses are removed
- **uniform:** bipolar time constants are all the same (no prox-distal difference)
- **no_mechs:** bipolar time constants are uniform, and GABA synapses are removed

In [ ]:
dsi_fig, dsi_ax = plt.subplots(len(dsis["control"]), sharex=True, figsize=(8, 8))
for (cond, ds) in dsis.items():
    for i, (a, d) in enumerate(zip(dsi_ax, ds.values())):
        lbl = {"label": cond} if not i else {}  # only add label once for each condition
        a.plot(velocities, np.mean(d, axis=0), linestyle="--", marker="o", **lbl)

for a, metric in zip(dsi_ax, dsis["control"].keys()):
    a.set_ylabel("%s DSi" % metric)
    a.set_ylim(0)
    clean_axes(a)

dsi_ax[-1].set_xlabel("Velocity (mm/s)")
dsi_fig.legend()
dsi_fig.show()

### Highlight of the thresholded area metric

In [ ]:
area_dsi_fig, area_dsi_ax = plt.subplots(1, figsize=(8, 4))
for (cond, ds) in dsis.items():
    area_dsi_ax.plot(
        velocities[1:-1], 
        np.mean(ds["thresh_area"][:, 1:-1], axis=0),
        linestyle="--",
        marker="o",
        label=cond
    )

area_dsi_ax.set_title("Tuning of Vm area above -55mV threshold")
area_dsi_ax.set_ylabel("PN DSi")
area_dsi_ax.set_xlabel("Velocity (mm/s)")
area_dsi_ax.set_ylim(0)
clean_axes(area_dsi_ax)
area_dsi_fig.legend()

area_dsi_fig.show()

### Terminal membrane voltage and (proximal) GABA conductance
Trial averages of membrane voltage (recorded at the distal tip) for each membrane, along with the GABA conductance they recieved proximal to their soma from the opposing SAC. Legend indicates the velocity (in mm/s) corresponding to each mean recording presented.

A figure is generated for each of the experimental conditions.

In [ ]:
alpha = .8
rec_figs = {}
for cond, ex in exps.items():
    fig, ax = plt.subplots(3, 2, sharex="col", sharey="row",figsize=(8, 8))
    rec_figs[cond] = {"fig": fig, "ax": ax}
    
    for vel, v_a, v_b, ica_a, ica_b, g_a, g_b in zip(
        velocities,
        np.mean(ex["term"]["a"]["v"], axis=0), 
        np.mean(ex["term"]["b"]["v"], axis=0),
        np.mean(ex["term"]["a"]["ica"], axis=0), 
        np.mean(ex["term"]["b"]["ica"], axis=0),
        np.mean(ex["gaba"]["a"]["g"], axis=0),
        np.mean(ex["gaba"]["b"]["g"], axis=0)
    ):
        ax[0][0].plot(rec_xaxis, v_a, alpha=alpha)
        ax[0][1].plot(rec_xaxis, v_b, alpha=alpha)
        ax[1][0].plot(rec_xaxis, ica_a, alpha=alpha)
        ax[1][1].plot(rec_xaxis, ica_b, alpha=alpha)
        ax[2][0].plot(rec_xaxis, g_a, alpha=alpha)
        ax[2][1].plot(rec_xaxis, g_b, label="%.2f mm/s" % vel, alpha=alpha)

    fig.suptitle(cond)
    ax[0][0].set_title("Sac A")
    ax[0][1].set_title("Sac B")
    ax[0][0].set_ylabel("Terminal Voltage (mV)")
    ax[1][0].set_ylabel("Terminal iCa (mA/cm2)")
    ax[2][0].set_ylabel("GABA Conducatance (μS)")
    ax[1][0].set_xlabel("Time (ms)")
    ax[1][1].set_xlabel("Time (ms)")
    fig.legend()
    fig.show()